In [1]:
import numpy as np
import pandas as pd
import os, string

def shorten(name):
    items = name.split("-")
    if len(items[0]) <= 3:
        return items[0]
#         if not items[1:]:
#             return items[0]
#         else:
#             return items[0]+"-"+reduce(lambda i, j: i+j, map(lambda i: i[0], items[1:]))
    else:
        return reduce(lambda i, j: i+j, map(lambda i: i[0], items))

In [2]:
file_list = sorted([file for root, subs, files in os.walk("../data/raw/investing") for file in files if file.endswith(".csv")])

In [3]:
tables = {}
for file in file_list:
    tables[string.join(file.replace(".csv", "").split("-")[:-1], "-")] = pd.read_csv("../data/raw/investing/{}".format(file), index_col="Datetime").sort_index()

In [4]:
kw, table = tables.popitem()
table.columns = ["{}-{}".format(col, shorten(kw)) for col in table.columns]
while tables:
    try:
        kw, other = tables.popitem()
        other.columns = ["{}-{}".format(col, shorten(kw)) for col in other.columns]
        table = table.join(other, how="outer")
    except:
        break

table.index = pd.DatetimeIndex(table.index)

In [5]:
raw_table = table.loc[table.index>"2000-01-01",:]
raw_table = raw_table.dropna(how="all")
raw_table.describe()

,Quarter-gdp,Actual-gdp,Forecast-gdp,Previous-gdp,Actual-adp,Forecast-adp,Previous-adp,Actual-ur,Forecast-ur,Previous-ur,...,Previous-ahe,Actual-ird,Forecast-ird,Previous-ird,Actual-cc,Forecast-cc,Previous-cc,Actual-cb,Forecast-cb,Previous-cb
count,45.000000,121.000000,121.000000,121.000000,202.000000,122.000000,202.000000,218.000000,117.000000,219.000000,...,143.000000,120.000000,80.000000,120.000000,218.000000,123.000000,219.000000,218.000000,122.000000,218.000000
mean,2.600000,1.684298,1.676033,1.646281,74762.376238,83344.262295,75118.811881,6.099083,7.032479,6.090411,...,0.195804,1.510417,0.512500,1.541667,0.165138,0.165041,0.165297,88.540826,76.574590,88.733028
std,1.095445,2.209638,2.132722,2.225947,189862.146823,191660.114558,192457.337115,1.767030,1.944880,1.769375,...,0.155135,1.715413,0.513889,1.753548,0.090443,0.049562,0.089244,26.046326,24.869024,25.965449
min,1.000000,-6.300000,-6.600000,-6.400000,-742000.000000,-660000.000000,-708000.000000,3.800000,4.000000,3.800000,...,-0.200000,0.250000,0.250000,0.250000,-0.100000,0.100000,-0.100000,25.000000,27.800000,25.000000
25%,2.000000,1.100000,1.000000,0.900000,-2250.000000,44000.000000,2000.000000,4.700000,5.100000,4.700000,...,0.100000,0.250000,0.250000,0.250000,0.100000,0.100000,0.100000,66.450000,55.850000,67.025000
50%,3.000000,2.100000,2.200000,2.000000,138000.000000,165500.000000,135000.000000,5.600000,7.200000,5.600000,...,0.200000,0.500000,0.250000,0.500000,0.200000,0.200000,0.200000,92.750000,72.700000,93.100000
75%,4.000000,2.900000,3.000000,3.000000,199750.000000,194000.000000,200500.000000,7.375000,8.900000,7.350000,...,0.300000,2.250000,0.500000,2.312500,0.200000,0.200000,0.200000,105.675000,96.000000,105.675000
max,4.000000,5.900000,5.900000,5.900000,343000.000000,230000.000000,343000.000000,10.200000,10.200000,10.200000,...,0.600000,6.500000,2.500000,6.500000,0.400000,0.300000,0.400000,144.700000,131.000000,144.700000


In [6]:
# save master
master_table = raw_table.interpolate(method="time")
master_table.reset_index().to_excel("../data/raw/investing/economic-indicators-master.xlsx", index=False)

# save actual
actual_table = raw_table.filter(like="Actual")
actual_table.columns = [col.replace("Actual-", "") for col in actual_table.columns]
actual_table = actual_table.dropna(how="all")
actual_table = actual_table.interpolate(method="time")
actual_table.insert(loc=0, column="Quarter-gdp", value=raw_table.get("Quarter-gdp"))
actual_table.reset_index().to_excel("../data/raw/investing/economic-indicators-actual.xlsx", index=False)

# save master (financial + economical indicators)
